# Reliability Engineering
### FT Chart Calculation

### Calculate the system reliability and unreliability from FT chart

$R_i\cdots$ reliability of an element or event<br>
$F_i\cdots$ unreliability of an element or event

### Formulas
#### In "series system" $R_s = \prod_i^n R_i$
> $R_s = R_1 \cdot R_2$<br>
$F_s = F_1 + F_2 - F_1 \cdot F_2$<br>

#### In "parallel system" $F_s = \prod_i^n F_i$
> $R_s = R_1 + R_2 - R_1 \cdot R_2$ <br>
$F_s = F_1 \cdot F_2$

In [2]:
from math import*

In [3]:
# create event class
class Event:
    """
        argument: event(unreliability "F", is_fault=0)
        is_fault --- success==0==False, fault==1==True
    """ 
    def __init__(self, unreliability, is_fault=0):
        self._F = unreliability
        self._R = 1 - unreliability
        self._is_fault = is_fault
    
    @property
    def F(self):
        return self._F
    @F.setter
    def F(self, unreliability):
        self._F = unreliability
        self._R = 1 - unreliability
        return
    
    @property
    def R(self):
        return self._R
    @R.setter
    def R(self, reliability):
        self._R = reliability
        self._F = 1 - reliability
        return
    
    @property
    def is_fault(self):
        return self._is_fault
    @is_fault.setter
    def is_fault(self, is_fault):
        self._is_fault = is_fault
        return
    
    
    
    
    # operator overload
    def __add__(self, other):
        R_new = self.R * other.R
        is_fault_new = 1 - (1 - self.is_fault) * (1 - other.is_fault) 
        return Event(1 - R_new, is_fault_new) # Event(unreliability, is_fault)
     
    def __mul__(self, other):
        F_new = self.F * other.F
        is_fault_new = self.is_fault * other.is_fault
        return Event(F_new, is_fault_new)
    
    def __str__(self):
        return "F = {F}, R = {R}".format(F=self.F, R=self.R)

#### Event class でできること
> - 不信頼度、信頼度の計算
> - 構造関数の値の計算

#### Event classの使い方
Instanciate element events and just calculate.<br>
各要素の不信頼度Fを求めて、それを引数にEvent Objectをつくって
ブール代数表記すれば、そのまま計算してくれるよ。<br>
$"*"$ as AND<br> $"+"$ as OR
>`XA = Event(unreliability=0.1)`<br>
 `XB = Event(unreliability=0.2)`<br>
 `XC = Event(unreliability=0.3)`<br>
 `T = (XA * XB) * (XA + XC)`


Example 1

In [4]:
# define each element XA to XC
XA = Event(0.1) # unreliability is 0.1
XB = Event(0.2) # unreliability is 0.2
XC = Event(0.3) # unreliability is 0.3
print(XA)
print(XB)
print(XC)

F = 0.1, R = 0.9
F = 0.2, R = 0.8
F = 0.3, R = 0.7


In [5]:
A = XA * XB
B = XA + XC
T = A * B
print(T)

F = 0.007400000000000001, R = 0.9926


Example 2

In [6]:
# define each unreliability F of an element
FM = 1 - exp(-2e-6*10000)
FS = 1 - exp(-3e-6*10000)
FE = 1 - exp(-5e-7*10000)
FB = 1 - exp(-5e-7*10000)
FG = 1 - exp(-5e-6*10000)

In [7]:
# definte each event of an element
M = Event(FM)
S = Event(FS)
E = Event(FE)
B = Event(FB)
G = Event(FG)

In [8]:
# T is the top event
T = M + (S + (E + (B + G)))
print(T)

F = 0.10416586470347167, R = 0.8958341352965283


Example 3

In [9]:
# 構造関数の計算
# こんな感じに状態表の一部も作れちゃうよん
X1 = Event(0, is_fault=0)
X2 = Event(0, is_fault=0)
X3 = Event(0, is_fault=1)
X4 = Event(0, is_fault=0)
for i in (0, 1):
    for j in (0, 1):
        for k in (0, 1):
            for l in (0, 1):
                X1.is_fault = l
                X2.is_fault = k
                X3.is_fault = j
                X4.is_fault = i
                T = (X1+X2) * (X1+X3+X4) # ブール代数表記をすると頂上事象Tの構造関数の値がオブジェクトに保存される。
                print("({},{},{},{})  top event = {}".format(l, k, j, i, T.is_fault))

(0,0,0,0)  top event = 0
(1,0,0,0)  top event = 1
(0,1,0,0)  top event = 0
(1,1,0,0)  top event = 1
(0,0,1,0)  top event = 0
(1,0,1,0)  top event = 1
(0,1,1,0)  top event = 1
(1,1,1,0)  top event = 1
(0,0,0,1)  top event = 0
(1,0,0,1)  top event = 1
(0,1,0,1)  top event = 1
(1,1,0,1)  top event = 1
(0,0,1,1)  top event = 0
(1,0,1,1)  top event = 1
(0,1,1,1)  top event = 1
(1,1,1,1)  top event = 1


In [10]:
# 手計算で作った頂上事象の構造関数の値と比較。
# 試しに上記のブール代数で表されるシステムの構造関数を作ってみた。
# define structure function
# phi1 and phi2 are the same structure function but in different expression
phi1 = lambda X1, X2, X3, X4 : (1 - (1 - X1) * (1 - X2)) * (1 - (1 - X1)*(1 - X3) * (1 - X4))
phi2 = lambda X1, X2, X3, X4 : X1 + X2*X3 + X2*X4 - X1*X2*X3 - X2*X3*X4 - X1*X2*X4 + X1*X2*X3*X4# 1 + X2 - X3 - X4 - X1*X2 + X1*X3 + X1*X4 + X3*X4 - X1*X3*X4

In [12]:
# ちゃんと一致した。
print("structure function phi1")
for i in (0, 1):
    for j in (0, 1):
        for k in (0, 1):
            for l in (0, 1):
                X1.is_fault = l
                X2.is_fault = k
                X3.is_fault = j
                X4.is_fault = i
                Fault = phi1(X1.is_fault, X2.is_fault, X3.is_fault, X4.is_fault)
                print("X = ({a},{b},{c},{d}), top event = {s}".format(a=l, b=k, c=j, d=i, s=Fault))
print()
print("structure function phi2")
for i in (0, 1):
    for j in (0, 1):
        for k in (0, 1):
            for l in (0, 1):
                X1.is_fault = l
                X2.is_fault = k
                X3.is_fault = j
                X4.is_fault = i
                Fault = phi2(X1.is_fault, X2.is_fault, X3.is_fault, X4.is_fault)
                print("X = ({a},{b},{c},{d}), top event = {s}".format(a=l, b=k, c=j, d=i, s=Fault))

structure function phi1
X = (0,0,0,0), top event = 0
X = (1,0,0,0), top event = 1
X = (0,1,0,0), top event = 0
X = (1,1,0,0), top event = 1
X = (0,0,1,0), top event = 0
X = (1,0,1,0), top event = 1
X = (0,1,1,0), top event = 1
X = (1,1,1,0), top event = 1
X = (0,0,0,1), top event = 0
X = (1,0,0,1), top event = 1
X = (0,1,0,1), top event = 1
X = (1,1,0,1), top event = 1
X = (0,0,1,1), top event = 0
X = (1,0,1,1), top event = 1
X = (0,1,1,1), top event = 1
X = (1,1,1,1), top event = 1

structure function phi2
X = (0,0,0,0), top event = 0
X = (1,0,0,0), top event = 1
X = (0,1,0,0), top event = 0
X = (1,1,0,0), top event = 1
X = (0,0,1,0), top event = 0
X = (1,0,1,0), top event = 1
X = (0,1,1,0), top event = 1
X = (1,1,1,0), top event = 1
X = (0,0,0,1), top event = 0
X = (1,0,0,1), top event = 1
X = (0,1,0,1), top event = 1
X = (1,1,0,1), top event = 1
X = (0,0,1,1), top event = 0
X = (1,0,1,1), top event = 1
X = (0,1,1,1), top event = 1
X = (1,1,1,1), top event = 1
